In [5]:
import torch 
from time import time

import numpy as np
import pandas as pd
import torch.nn.functional as F
from tdmil.modelMIL import MILAttention, TransformerMIL

In [ ]:
def load_checkpoint(model, model_path):
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint["model_state_dict"])

In [7]:
attention_model = TransformerMIL(
            embed_dim=384,
            depth=4,
            num_heads=12,
        )
_ = attention_model.to("cuda")

In [8]:
load_checkpoint(attention_model, "model_weights/property_img_transformer_numpygenerator/checkpoint_00100.pt")

In [12]:
inp_csv = "datasets/val_image_emb.csv"
inp_df = pd.read_csv(inp_csv)
inp_df.sample(2, random_state=15)

,array_path,bag_length,target
29526,datasets/image_emb/ 4340 Valencia Dr ...,38,750000.0
16674,datasets/image_emb/ 8324 Black Pearl Ct ...,24,264364.0


In [86]:
max_bag_length = 45
array_path = inp_df.sample(2, random_state=15).iloc[0]["array_path"]

In [90]:
all_embeddings = []
all_arrays = []
all_torch_arrays = []
all_torch_masks = []
for _ in range(5):
    np_array = np.load(array_path)
    bag_length = len(np_array)
    indices = np.arange(bag_length)
    np.random.shuffle(indices)
    np_array = np_array[indices]
    all_arrays.append(np_array)
    attention_mask = torch.ones((max_bag_length), dtype=torch.float32)
    inp_torch_array = torch.zeros((max_bag_length, 384), dtype=torch.float32)
    inp_torch_array[:bag_length, :] = torch.tensor(np_array)
    attention_mask[bag_length:] = 0
    all_torch_arrays.append(inp_torch_array)
    all_torch_masks.append(attention_mask)
    inp_torch_array = inp_torch_array.unsqueeze_(0)
    attention_mask = attention_mask.unsqueeze_(0)
    inp_ = inp_torch_array.cuda(non_blocking=True)
    mask_ = attention_mask.cuda(non_blocking=True)
    with torch.no_grad():
        forward_result = attention_model(inp_, mask_, return_cls_token=True)
        batch_embedding = forward_result["cls_token"].clone().detach().cpu().numpy().astype(np.float32)
    all_embeddings.append(batch_embedding[0])

In [92]:
inp2_ = torch.concat(all_torch_arrays).cuda()
mask2_ = torch.concat(all_torch_masks).cuda()
with torch.no_grad():
    forward_result = attention_model(inp2_, mask2_, return_cls_token=True)
    batch_embedding2 = forward_result["cls_token"].clone().detach().cpu().numpy().astype(np.float32)


In [127]:
np.sum(batch_embedding2[0] != batch_embedding2[1])

0

In [94]:
batch_embedding2[0][0:20], batch_embedding2[1][0:20]

(array([-24.720535 , -52.081085 ,  41.67712  ,  15.15655  , -11.127977 ,
         82.48938  , -29.966606 , -31.782497 ,  12.296479 , -72.21116  ,
        -57.312946 , -10.943734 ,   1.1515127,  21.702269 ,  38.567696 ,
         58.712196 ,  47.17046  ,  35.95886  , -47.635925 ,   5.275736 ],
       dtype=float32),
 array([-24.720535 , -52.081085 ,  41.67712  ,  15.15655  , -11.127977 ,
         82.48938  , -29.966606 , -31.782497 ,  12.296479 , -72.21116  ,
        -57.312946 , -10.943734 ,   1.1515127,  21.702269 ,  38.567696 ,
         58.712196 ,  47.17046  ,  35.95886  , -47.635925 ,   5.275736 ],
       dtype=float32))

In [124]:
torch.sort(inp2_[0, 0:45, 1]).values != torch.sort(inp2_[2, 0:45, 1]).values

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False], device='cuda:0')

In [110]:
mask2_[4]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')

In [83]:
# all_torch_arrays[1][0][:, 0]